In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity



df = pd.read_csv("train.csv")
df.head()

In [ ]:
prompt = df["prompt"][0]
response_a = df["response_a"][0]
response_b = df["response_b"][0]

print(prompt)
print(response_a)
print(response_b)


In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"  # Good model for semantic similarity
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
def get_embedding(text, tokenizer, model, device):
    # Tokenize input
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    
    # Get model output
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    # Use the [CLS] token embedding as the sentence representation
    # or use mean pooling for a potentially better representation
    token_embeddings = model_output.last_hidden_state
    
    # Mean pooling - taking average of all token embeddings
    attention_mask = encoded_input['attention_mask']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    embedding = (sum_embeddings / sum_mask).cpu().numpy()
    
    return embedding[0]  # Return the first (and only) embedding

In [ ]:
# Get embeddings for prompt and response
prompt_embedding = get_embedding(prompt, tokenizer, model, device)
responsea_embedding = get_embedding(response_a, tokenizer, model, device)
responseb_embedding = get_embedding(response_b, tokenizer, model, device)

In [ ]:
# print(prompt_embedding)
# print(responsea_embedding)
# print(responseb_embedding)


In [17]:
similarity = cosine_similarity([prompt_embedding], [responsea_embedding])[0][0]

print(f"Similarity between prompt and response A: {similarity}")

similarity = cosine_similarity([prompt_embedding], [responseb_embedding])[0][0]

print(f"Similarity between prompt and response B: {similarity}")


Similarity between prompt and response A: 0.37852227687835693
Similarity between prompt and response B: 0.5427287817001343
